In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
import scipy
from numpy import array
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, merge, Activation 
from keras.layers import Conv2D, Concatenate, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D, SeparableConv2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from google.colab import files
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 12  # #layers_per_block(12) = (depth(40) - (total_blocks(3) + 1)) / total_blocks(3). It will further divided by 2 in case of DenseNet-BC"
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


In [0]:

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train_resized = array([scipy.misc.imresize(image, (24,24,3)) for image in x_train])
x_test_resized = array([scipy.misc.imresize(image, (24,24,3)) for image in x_test])
print(x_test_resized.shape)
# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


(10000, 24, 24, 3)


In [0]:
# Data Augmentation on CIFAR-10 (Source : https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images


datagen.fit(x_train)
datagen.fit(x_train_resized)

In [0]:
# Dense Block with RMSA kernel Initializer
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):

# Bottlenack layer with compression in dense block. Accuracy : 81.21

#         BatchNorm = BatchNormalization()(temp)
#         relu = Activation('relu')(BatchNorm)
#         Conv2D_1_1 = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='valid', kernel_initializer='VarianceScaling')(relu)
#         if dropout_rate>0:
#           Conv2D_1_1 = Dropout(dropout_rate)(Conv2D_1_1)
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)

# Tried Depthwise Separable Convolution in place of Conv2D. Accuracy : 71.43

        # Removed Compression from 3*3 Conv2D.
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same', kernel_initializer='VarianceScaling')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same', kernel_initializer='VarianceScaling')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2),strides=2)(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = GlobalAveragePooling2D()(relu)
    #flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(AvgPooling)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(None, None, channel,))
First_Conv2D = Conv2D(2*num_filter, (3,3), use_bias=False ,padding='same', kernel_initializer='VarianceScaling')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, None, None, 4 1296        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, None, None, 4 192         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, None, None, 4 0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# Training on 24*24 Images for 15 epochs
model.fit_generator(datagen.flow(x_train_resized, y_train,
                    batch_size=batch_size),
                    epochs=15,
                    verbose=1,
                    validation_data=(x_test_resized, y_test))
model.save_weights("Dense_model_1.h5")


Epoch 1/15
782/782 [==============================] - 277s 354ms/step - loss: 1.6119 - acc: 0.3996 - val_loss: 1.6135 - val_acc: 0.4536
Epoch 2/15
122/782 [===>..........................] - ETA: 3:33 - loss: 1.3738 - acc: 0.4946

782/782 [==============================] - 262s 335ms/step - loss: 1.2960 - acc: 0.5277 - val_loss: 1.7809 - val_acc: 0.4870
Epoch 3/15
214/782 [=======>......................] - ETA: 3:03 - loss: 1.1528 - acc: 0.5874

782/782 [==============================] - 263s 336ms/step - loss: 1.1265 - acc: 0.5971 - val_loss: 2.4729 - val_acc: 0.4491
Epoch 4/15
248/782 [========>.....................] - ETA: 2:52 - loss: 1.0384 - acc: 0.6316

782/782 [==============================] - 262s 335ms/step - loss: 1.0196 - acc: 0.6375 - val_loss: 2.0286 - val_acc: 0.5055
Epoch 5/15
261/782 [=========>....................] - ETA: 2:47 - loss: 0.9652 - acc: 0.6572

782/782 [==============================] - 260s 333ms/step - loss: 0.9462 - acc: 0.6636 - val_loss: 1.0821 - val_acc: 0.6495
Epoch 6/15
266/782 [=========>....................] - ETA: 2:45 - loss: 0.9020 - acc: 0.6800

...

782/782 [==============================] - 262s 335ms/step - loss: 0.7030 - acc: 0.7540 - val_loss: 0.8064 - val_acc: 0.7411
Epoch 12/15
267/782 [=========>....................] - ETA: 2:45 - loss: 0.6737 - acc: 0.7646

782/782 [==============================] - 262s 335ms/step - loss: 0.6803 - acc: 0.7639 - val_loss: 0.7552 - val_acc: 0.7617
Epoch 13/15
267/782 [=========>....................] - ETA: 2:45 - loss: 0.6521 - acc: 0.7729

782/782 [==============================] - 260s 333ms/step - loss: 0.6567 - acc: 0.7720 - val_loss: 0.8421 - val_acc: 0.7305
Epoch 14/15
267/782 [=========>....................] - ETA: 2:45 - loss: 0.6378 - acc: 0.7769

782/782 [==============================] - 261s 333ms/step - loss: 0.6409 - acc: 0.7765 - val_loss: 0.9796 - val_acc: 0.7164
Epoch 15/15
267/782 [=========>....................] - ETA: 2:44 - loss: 0.6173 - acc: 0.7875

782/782 [==============================] - 261s 334ms/step - loss: 0.6244 - acc: 0.7834 - val_loss: 0.8118 - val_acc: 0.7581


In [0]:
from google.colab import files
files.download('Dense_model_1.h5')

In [0]:
model.load_weights('Dense_model_1.h5')
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("Dense_model_2.h5")


Epoch 1/5
782/782 [==============================] - 383s 490ms/step - loss: 0.5687 - acc: 0.8028 - val_loss: 2.1841 - val_acc: 0.3905
Epoch 2/5
127/782 [===>..........................] - ETA: 5:03 - loss: 0.5441 - acc: 0.8135

782/782 [==============================] - 382s 488ms/step - loss: 0.5406 - acc: 0.8133 - val_loss: 0.8681 - val_acc: 0.7528
Epoch 3/5
216/782 [=======>......................] - ETA: 4:23 - loss: 0.5301 - acc: 0.8181

782/782 [==============================] - 382s 489ms/step - loss: 0.5262 - acc: 0.8183 - val_loss: 0.7422 - val_acc: 0.7755
Epoch 4/5
250/782 [========>.....................] - ETA: 4:07 - loss: 0.4964 - acc: 0.8273

782/782 [==============================] - 382s 488ms/step - loss: 0.5036 - acc: 0.8246 - val_loss: 0.8276 - val_acc: 0.7581
Epoch 5/5
262/782 [=========>....................] - ETA: 4:01 - loss: 0.4958 - acc: 0.8288

782/782 [==============================] - 382s 489ms/step - loss: 0.4975 - acc: 0.8279 - val_loss: 0.9713 - val_acc: 0.7270


In [0]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("Dense_model_3.h5")


Epoch 1/10
782/782 [==============================] - 385s 493ms/step - loss: 0.4473 - acc: 0.8434 - val_loss: 0.8056 - val_acc: 0.7678
Epoch 2/10
122/782 [===>..........................] - ETA: 5:09 - loss: 0.4650 - acc: 0.8371

782/782 [==============================] - 385s 492ms/step - loss: 0.4456 - acc: 0.8454 - val_loss: 0.8463 - val_acc: 0.7653
Epoch 3/10
213/782 [=======>......................] - ETA: 4:26 - loss: 0.4456 - acc: 0.8446

782/782 [==============================] - 385s 492ms/step - loss: 0.4398 - acc: 0.8466 - val_loss: 0.7191 - val_acc: 0.7819
Epoch 4/10
247/782 [========>.....................] - ETA: 4:11 - loss: 0.4256 - acc: 0.8522

782/782 [==============================] - 386s 493ms/step - loss: 0.4305 - acc: 0.8512 - val_loss: 0.6535 - val_acc: 0.8134
Epoch 5/10
260/782 [========>.....................] - ETA: 4:04 - loss: 0.4132 - acc: 0.8576

782/782 [==============================] - 385s 493ms/step - loss: 0.4214 - acc: 0.8547 - val_loss: 0.7150 - val_acc: 0.7887
Epoch 6/10
265/782 [=========>....................] - ETA: 4:02 - loss: 0.4155 - acc: 0.8571

782/782 [==============================] - 385s 493ms/step - loss: 0.4160 - acc: 0.8555 - val_loss: 0.5748 - val_acc: 0.8327
Epoch 7/10
267/782 [=========>....................] - ETA: 4:02 - loss: 0.4155 - acc: 0.8519

782/782 [==============================] - 386s 493ms/step - loss: 0.4106 - acc: 0.8548 - val_loss: 0.9513 - val_acc: 0.7482
Epoch 8/10
268/782 [=========>....................] - ETA: 4:00 - loss: 0.4072 - acc: 0.8607

782/782 [==============================] - 386s 493ms/step - loss: 0.4065 - acc: 0.8600 - val_loss: 0.8104 - val_acc: 0.7750
Epoch 9/10
268/782 [=========>....................] - ETA: 4:01 - loss: 0.3988 - acc: 0.8605

782/782 [==============================] - 385s 493ms/step - loss: 0.3987 - acc: 0.8615 - val_loss: 0.6980 - val_acc: 0.7967
Epoch 10/10
267/782 [=========>....................] - ETA: 4:00 - loss: 0.3879 - acc: 0.8663

782/782 [==============================] - 384s 491ms/step - loss: 0.3936 - acc: 0.8644 - val_loss: 0.5786 - val_acc: 0.8301


In [0]:
from google.colab import files
files.download('Dense_model_3.h5')

In [0]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("Dense_model_4.h5")


Epoch 1/5
782/782 [==============================] - 381s 487ms/step - loss: 0.3199 - acc: 0.8886 - val_loss: 0.4828 - val_acc: 0.8594
Epoch 2/5
127/782 [===>..........................] - ETA: 5:04 - loss: 0.3229 - acc: 0.8848

782/782 [==============================] - 382s 488ms/step - loss: 0.3218 - acc: 0.8888 - val_loss: 0.4854 - val_acc: 0.8541
Epoch 3/5
216/782 [=======>......................] - ETA: 4:22 - loss: 0.3131 - acc: 0.8893

782/782 [==============================] - 381s 487ms/step - loss: 0.3256 - acc: 0.8859 - val_loss: 0.7244 - val_acc: 0.8090
Epoch 4/5
250/782 [========>.....................] - ETA: 4:05 - loss: 0.3114 - acc: 0.8909

782/782 [==============================] - 380s 486ms/step - loss: 0.3138 - acc: 0.8904 - val_loss: 0.6021 - val_acc: 0.8364
Epoch 5/5
262/782 [=========>....................] - ETA: 4:00 - loss: 0.3209 - acc: 0.8873

782/782 [==============================] - 380s 486ms/step - loss: 0.3150 - acc: 0.8905 - val_loss: 0.4601 - val_acc: 0.8652


In [0]:
from google.colab import files
files.download('Dense_model_4.h5')

In [0]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("Dense_model_5.h5")


Epoch 1/10
782/782 [==============================] - 380s 486ms/step - loss: 0.3078 - acc: 0.8934 - val_loss: 0.6325 - val_acc: 0.8276
Epoch 2/10
122/782 [===>..........................] - ETA: 5:03 - loss: 0.2951 - acc: 0.8942

782/782 [==============================] - 380s 485ms/step - loss: 0.3086 - acc: 0.8933 - val_loss: 0.5372 - val_acc: 0.8463
Epoch 3/10
213/782 [=======>......................] - ETA: 4:22 - loss: 0.3055 - acc: 0.8946

782/782 [==============================] - 379s 485ms/step - loss: 0.3074 - acc: 0.8936 - val_loss: 0.5322 - val_acc: 0.8489
Epoch 4/10
247/782 [========>.....................] - ETA: 4:05 - loss: 0.2945 - acc: 0.8968

782/782 [==============================] - 379s 485ms/step - loss: 0.3040 - acc: 0.8942 - val_loss: 0.4872 - val_acc: 0.8628
Epoch 5/10
260/782 [========>.....................] - ETA: 4:00 - loss: 0.2973 - acc: 0.8963

782/782 [==============================] - 379s 485ms/step - loss: 0.3012 - acc: 0.8956 - val_loss: 0.6657 - val_acc: 0.8241
Epoch 6/10
265/782 [=========>....................] - ETA: 3:58 - loss: 0.2922 - acc: 0.8987

782/782 [==============================] - 388s 496ms/step - loss: 0.2977 - acc: 0.8967 - val_loss: 0.5717 - val_acc: 0.8381
Epoch 7/10
267/782 [=========>....................] - ETA: 3:57 - loss: 0.2976 - acc: 0.8958

782/782 [==============================] - 379s 484ms/step - loss: 0.2995 - acc: 0.8954 - val_loss: 0.4692 - val_acc: 0.8637
Epoch 8/10
268/782 [=========>....................] - ETA: 3:57 - loss: 0.2875 - acc: 0.8995

782/782 [==============================] - 380s 486ms/step - loss: 0.2932 - acc: 0.8987 - val_loss: 0.4736 - val_acc: 0.8664
Epoch 9/10
268/782 [=========>....................] - ETA: 3:56 - loss: 0.2858 - acc: 0.8991

782/782 [==============================] - 380s 486ms/step - loss: 0.2917 - acc: 0.8971 - val_loss: 0.4415 - val_acc: 0.8703
Epoch 10/10
267/782 [=========>....................] - ETA: 3:57 - loss: 0.2827 - acc: 0.9004

782/782 [==============================] - 380s 486ms/step - loss: 0.2913 - acc: 0.8977 - val_loss: 0.5547 - val_acc: 0.8450


In [0]:
early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,callbacks=[early],
                    validation_data=(x_test, y_test))
model.save_weights("Dense_model_6.h5")


Epoch 1/10
782/782 [==============================] - 380s 486ms/step - loss: 0.2871 - acc: 0.9000 - val_loss: 0.5546 - val_acc: 0.8443
Epoch 2/10
122/782 [===>..........................] - ETA: 5:05 - loss: 0.2752 - acc: 0.9043

782/782 [==============================] - 380s 486ms/step - loss: 0.2827 - acc: 0.9013 - val_loss: 0.4481 - val_acc: 0.8744
Epoch 3/10
213/782 [=======>......................] - ETA: 4:23 - loss: 0.2791 - acc: 0.9013

782/782 [==============================] - 380s 486ms/step - loss: 0.2833 - acc: 0.9019 - val_loss: 0.4188 - val_acc: 0.8761
Epoch 4/10
247/782 [========>.....................] - ETA: 4:07 - loss: 0.2788 - acc: 0.9013

591/782 [==============================] - 381s 486ms/step - loss: 0.2839 - acc: 0.9017 - val_loss: 0.4348 - val_acc: 0.8754


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 23s 2ms/step
Test loss: 0.4347574817597866
Test accuracy: 0.8754


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_final.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
files.download('DNST_model_final.h5')